In [1]:
from tqdm import tqdm
from baysic.structure_evaluation import point_energy, relaxed_energy
from baysic.utils import json_to_df
import pandas as pd

df = pd.read_pickle('merged_test_data3.pkl')

In [2]:
energies = []

for i in tqdm(df.index[:20]):
    energies.append(point_energy(df.loc[i, 'struct']))

100%|██████████| 20/20 [00:01<00:00, 13.79it/s]


- ~14 it/s for Chrom

In [2]:
from copy import deepcopy
from tqdm import tqdm

from baysic.structure_evaluation import relaxed_energy
from baysic.utils import df_to_json, json_to_df

df = json_to_df('CeBRh3_raw.json')

best_relaxed = []
best_e_form = []

for struct in tqdm(df['gen']):
    rel, eform = relaxed_energy(deepcopy(struct), long=True)
    best_relaxed.append(rel)
    best_e_form.append(eform)

df['best_relaxed'] = best_relaxed
df['best_e_form'] = best_e_form

df_to_json(df, 'CeBRh3.json')

  0%|          | 6/1526 [00:04<19:11,  1.32it/s]


KeyboardInterrupt: 

- ~1 it/s for Chrom

In [5]:
import matgl
from matgl.ext.ase import M3GNetCalculator, MolecularDynamics, Relaxer
from pymatgen.core import Structure


eform = matgl.load_model('M3GNet-MP-2018.6.1-Eform')
pot = matgl.load_model("M3GNet-MP-2021.2.8-PES")
relaxer = Relaxer(potential=pot)

def relaxed_energy(struct: Structure, long: bool = False) -> (Structure, float):
    if long:
        params = dict(fmax=0.01, steps=300)    
    else:
        params = dict(fmax=0.02, steps=5)
        
    relax_results = relaxer.relax(struct, **params)
    # extract results
    final_structure = relax_results["final_structure"]
    final_energy = relax_results["trajectory"].energies[-1]
    # print out the final relaxed structure and energy
    return (final_structure, final_energy / final_structure.num_sites)

relaxed_energy(df['gen'].iloc[0], long=True)

(Structure Summary
 Lattice
     abc : 4.244907169529787 4.244907014377643 4.244907209074851
  angles : 89.99999985433685 89.99999969904897 90.00000001363075
  volume : 76.48998572940963
       A : 4.244907169529787 -5.049345516923652e-10 1.1148370774176157e-08
       B : -5.049347875189044e-10 4.244907014377643 5.395917569282424e-09
       C : 1.114837050085371e-08 5.3959174748069565e-09 4.244907209074851
     pbc : True True True
 PeriodicSite: Rh (5.973e-09, 2.122, 2.122) [1.535e-10, 0.5, 0.5]
 PeriodicSite: Rh (2.122, 3.741e-09, 2.122) [0.5, 3.052e-10, 0.5]
 PeriodicSite: Rh (2.122, 2.122, 1.012e-08) [0.5, 0.5, 4.348e-10]
 PeriodicSite: Ce (5.107e-10, 5.369e-10, 1.163e-09) [1.203e-10, 1.265e-10, 2.74e-10]
 PeriodicSite: B (2.122, 2.122, 2.122) [0.5, 0.5, 0.5],
 -7.7015228271484375)

In [ ]:
from copy import deepcopy
from tqdm import tqdm

from baysic.structure_evaluation import relaxed_energy
from baysic.utils import df_to_json, json_to_df

df = json_to_df('CeBRh3_raw.json')

best_relaxed = []
best_e_form = []

for struct in tqdm(df['gen']):
    rel, eform = relaxed_energy(deepcopy(struct), long=True)
    best_relaxed.append(rel)
    best_e_form.append(eform)

df['best_relaxed'] = best_relaxed
df['best_e_form'] = best_e_form

df_to_json(df, 'CeBRh3.json')